# Distributed Applications REST and SOAP test
## Libraries

In [1]:
!pip install pandas
!pip install aiocurl
#IF ERROR: sudo apt install libcurl4-openssl-dev libssl-dev

import asyncio
import nest_asyncio

import numpy as np
import pandas as pd

import os
import sys
import requests

import math
import time
import threading 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utility Functions

### Open Servers

In [2]:
port = 8083

def startEUServer():
    psw = 'distributed@10'
    vm = 'yonira@yonira.westeurope.cloudapp.azure.com'
    cmd = 'java -jar -Dspring.profiles.active=prod rest.jar'
    os.system('sshpass -p ' + psw + ' ssh -tt ' + vm + ' "' + cmd + '"')

def startJAPServer():
    psw = 'distributed@10'
    vm = 'yonira@super.japaneast.cloudapp.azure.com'
    cmd = 'java -jar -Dspring.profiles.active=prod rest.jar'
    os.system('sshpass -p ' + psw + ' ssh -tt ' + vm + ' "' + cmd + '"')
    
    
def startUSWServer():
    psw = 'Haslo1234567890'
    vm = 'filip@filip.westus3.cloudapp.azure.com'
    cmd = 'java -jar -Dspring.profiles.active=prod rest.jar'
    os.system('sshpass -p ' + psw + ' ssh -tt ' + vm + ' "' + cmd + '"')

def startUSEServer():
    psw = 'DistributedApplications1'
    vm = 'thant@thant.eastus.cloudapp.azure.com'
    cmd = 'java -jar -Dspring.profiles.active=prod rest.jar'
    os.system('sshpass -p ' + psw + ' ssh -tt ' + vm + ' "' + cmd + '"')

#startEUServer()
#url = "http://yonira.westeurope.cloudapp.azure.com:" + str(port) + "/restrpc/meals"

#startJAPServer()
url = "http://super.japaneast.cloudapp.azure.com:" + str(port) + "/restrpc/meals"

#startUSWServer()
#url = "http://filip.westus3.cloudapp.azure.com:" + str(port) + "/restrpc/meals"

#startUSEServer()
#url = "http://thant.eastus.cloudapp.azure.com:" + str(port) + "/restrpc/meals"

### RestRequestAsync method

Makes a request to the given URL


In [3]:
async def restRequestAsync(url, requestNumPerTraffic) -> None:
    try:
        response = requests.get(url)
        if (response.status_code != 200):
            print('BAD', end = " ")
        else:
            print('OK', end = " ")
        
    except Exception as e:
        raise 'REQUEST ERR'

### BackgroundRequest method
Creates tasks that run restRequestAsync method continuously

In [4]:
async def backgroundRequest(url, requestNum):
    print("\nTESTING WITH " + str(requestNum) + " REQUESTS:")
    background_tasks = set()
    
    for i in range(requestNum):
        task = asyncio.create_task(restRequestAsync(url, requestNum))
        background_tasks.add(task)
        task.add_done_callback(background_tasks.discard)

### Asynchronous Traffic and Request Generator

Generates asynchronous traffic and request to a server in the background. It creates a task for making requests and stores the results in an array, and another task for sending traffic requests. The method then waits for a specified period before making the next request.

In [5]:
async def main(requestNum, requestNumMultiplier, requestNumPerTraffic, url, resultsArray):
    counter = 0
    exponent = 0
    for i in range(requestNum):
        try:
            start = time.time()
            requestTask = asyncio.create_task(restRequestAsync(url, requestNumMultiplier ** exponent))
            trafficTask = asyncio.create_task(backgroundRequest(url, requestNumMultiplier ** exponent))
            await asyncio.wait_for(requestTask, 10000)
            end = time.time()
            
            resultsArray[i, 0] =  requestNumMultiplier ** exponent
            resultsArray[i, 1] =  end - start
            print("\nTime Elapsed:", end - start)
            
            counter += 1
            if counter >= requestNumPerTraffic: 
                counter = 0
                exponent += 1
        
        except Exception as e:
            print(f"Could not connect")
            print(e)
            break


# TEST
Define the parameters for the test and run it.

In [6]:
requestNumPerTraffic = 3
requestNumMultiplier = 2
requestNum = 24
resultsArray = np.zeros((requestNum, 2))

nest_asyncio.apply()
asyncio.run(main(requestNum, requestNumMultiplier, requestNumPerTraffic, url, resultsArray))

OK 
TESTING WITH 1 REQUESTS:
OK 
Time Elapsed: 1.088261604309082
OK 
TESTING WITH 1 REQUESTS:
OK 
Time Elapsed: 0.989121675491333
OK 
TESTING WITH 1 REQUESTS:
OK 
Time Elapsed: 1.857489824295044
OK 
TESTING WITH 2 REQUESTS:
OK OK 
Time Elapsed: 2.4927029609680176
OK 
TESTING WITH 2 REQUESTS:
OK OK 
Time Elapsed: 1.4336352348327637
OK 
TESTING WITH 2 REQUESTS:
OK OK 
Time Elapsed: 2.8276736736297607
OK 
TESTING WITH 4 REQUESTS:
OK OK OK OK 
Time Elapsed: 4.432527542114258
OK 
TESTING WITH 4 REQUESTS:
OK OK OK OK 
Time Elapsed: 5.085401773452759
OK 
TESTING WITH 4 REQUESTS:
OK OK OK OK 
Time Elapsed: 5.935207843780518
OK 
TESTING WITH 8 REQUESTS:
OK OK OK OK OK OK OK OK 
Time Elapsed: 6.356433629989624
OK 
TESTING WITH 8 REQUESTS:
OK OK OK OK OK OK OK OK 
Time Elapsed: 4.696670770645142
OK 
TESTING WITH 8 REQUESTS:
OK OK OK OK OK OK OK OK 
Time Elapsed: 9.421898603439331
OK 
TESTING WITH 16 REQUESTS:
OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK 
Time Elapsed: 13.950438022613525
OK 
TE

## Print results

In [7]:
np.set_printoptions(precision = 4, suppress = True)
df = pd.DataFrame(resultsArray, columns = ['requests', 'time'])
df = df.groupby(['requests'])['time'].mean()
print(df.T)

requests
1.0       1.311624
2.0       2.251337
4.0       5.151046
8.0       6.825001
16.0     12.053461
32.0     22.526765
64.0     53.165197
128.0    95.685674
Name: time, dtype: float64
